# modules

In [1]:
import os

import numpy as np
import pandas as pd

# benchmark

In [2]:
def process_action(action):
    action = action.split('\n')
    content = {}
    content['alternatives'] = int(action[1].split('=')[1].strip())
    depths = [int(d) for d in action[2].split('=')[1].strip().split(' ')]
    content['mean_depth'] = float(np.mean(depths))
    content['mean_depth_std'] = float(np.std(depths))
    content['max_depth'] = int(action[3].split(',')[2].split('=')[1].strip())
    content['score'] = int(action[3].split(',')[1].split('=')[1].strip())
    return content

In [3]:
def process_log(log, h1, h2):
    content = {}
    log = log.split('\n\n')
    content['W'] = int(log[0].split(',')[0].split('=')[1].strip())
    content['B'] = int(log[0].split(',')[1].split('=')[1].strip())
    content['reduction'] = int(log[-1].split('\n')[0].split(',')[0].split('=')[1].strip())
    content['alpha'] = int(log[-1].split('\n')[0].split(',')[1].split('=')[1].strip())
    content['beta'] = int(log[-1].split('\n')[0].split(',')[2].split('=')[1].strip())
    content['time'] = float(log[-1].split('\n')[1].split(',')[0].split('=')[1].strip())
    content['W_time'] = float(log[-1].split('\n')[1].split(',')[1].split('=')[1].strip())
    content['B_time'] = float(log[-1].split('\n')[1].split(',')[2].split('=')[1].strip())
    content['winner'] = int(h1) if log[-2].split('=')[1].strip() == 'MAX' else int(h2)
    log = log[1:-3]
    content['moves'] = len(log)

    for i, action in enumerate(log):
        a = process_action(action)
        if i%2 == 0:
            t = 'W'
        else:
            t = 'B'
        if i==0 or i==1:
            content[f'{t}_alter'] = []
            content[f'{t}_mean_depth'] = []
            content[f'{t}_max_depth'] = []
            content[f'{t}_score'] = []
            
        content[f'{t}_alter'].append(a['alternatives'])
        content[f'{t}_mean_depth'].append(a['mean_depth'])
        content[f'{t}_max_depth'].append(a['max_depth'])
        content[f'{t}_score'].append(a['score'])
        
    # print(content)
            
    for t in ['W', 'B']:
        for s in ['alter', 'mean_depth', 'max_depth', 'score']:
            content[f'{t}_{s}'] = f'{round(float(np.mean(content[f'{t}_{s}'])))} +/- {round(float(np.std(content[f'{t}_{s}'])))}'
    return content

In [6]:
matchups = []
dir = 'logs/seq'
for fi in os.listdir(dir):
    with open(f"{dir}/{fi}") as f:
        log = f.read()
        h1, h2 = fi.split('_')
        content = process_log(log, h1, h2)
        matchups.append(content)

In [9]:
df = pd.DataFrame(matchups)
df = df.sort_values(by='W')

In [10]:
df.to_csv('data/benchmark.csv', index=False)

# Parallelization

In [3]:
dfp = pd.read_csv('data/parallel.csv')
dfp

,Heuristic,Parallel Depth,Execution Time,Acceleration,Pruning,Pruning Loss (%)
0,1,seq,286.3,NaN,4013442,NaN
1,1,1,56.0,NaN,658416,NaN
2,1,2,53.6,NaN,659202,NaN
3,1,3,57.3,NaN,657668,NaN
4,1,4,50.0,NaN,660487,NaN
5,3,seq,314.0,NaN,1268687,NaN
6,3,1,83.6,NaN,504238,NaN
7,3,2,83.6,NaN,504662,NaN
8,3,3,82.5,NaN,505492,NaN
9,3,4,78.7,NaN,505228,NaN


In [5]:
seq_times = dfp[dfp['Parallel Depth'] == 'seq'][['Heuristic', 'Execution Time']].set_index('Heuristic')
seq_times

,Execution Time
Heuristic,
1,286.3
3,314.0
4,195.2
5,119.4


In [7]:
dfp['Acceleration'] = dfp.apply(lambda row: seq_times.loc[row['Heuristic'], 'Execution Time'] / row['Execution Time'] if pd.notnull(row['Execution Time']) else None, axis=1)

In [9]:
prunings = dfp[dfp['Parallel Depth'] == 'seq'][['Heuristic', 'Pruning']].set_index('Heuristic')

In [15]:
dfp['Pruning Loss (%)'] = dfp.apply(lambda row: 100 * (1 - row['Pruning'] / prunings.loc[row['Heuristic'], 'Pruning']) if pd.notnull(row['Pruning']) else None, axis=1)

In [17]:
dfp.to_csv('data/parallel.csv', index=False)